In [ ]:
%pip install -q oqs matplotlib pandas

import sys, platform, time, os
from importlib.metadata import version, PackageNotFoundError

try:
    oqs_version = version("oqs")
except PackageNotFoundError:
    oqs_version = "unknown"

import oqs
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

print("Python:", sys.version.split()[0], "| OS:", platform.platform())
print("liboqs (python package) version:", oqs_version)


In [ ]:
import oqs, time, pandas as pd

CANDIDATE_KEMS = ["ML-KEM-512", "ML-KEM-768", "ML-KEM-1024", "KYBER512", "KYBER768", "KYBER1024"]
CANDIDATE_SIGS = ["ML-DSA-44", "ML-DSA-65", "DILITHIUM_2", "DILITHIUM_3"]

def available_kems():
    ok = []
    for name in CANDIDATE_KEMS:
        try:
            with oqs.KeyEncapsulation(name) as _:
                ok.append(name)
        except Exception:
            pass
    return ok

def available_sigs():
    ok = []
    for name in CANDIDATE_SIGS:
        try:
            with oqs.Signature(name) as _:
                ok.append(name)
        except Exception:
            pass
    return ok

KEMS = available_kems()
SIGS = available_sigs()
print("Available KEMs:", KEMS)
print("Available SIGs:", SIGS)

results = []
for kem in KEMS:
    with oqs.KeyEncapsulation(kem) as alice, oqs.KeyEncapsulation(kem) as bob:
        pk = alice.generate_keypair()
        t0 = time.perf_counter()
        ct, ss_bob = bob.encap_secret(pk)
        ss_alice = alice.decap_secret(ct)
        t1 = time.perf_counter()
        results.append(("KEM", kem, len(pk), len(ct), len(ss_bob), (t1 - t0) * 1e3))

for sig in SIGS:
    with oqs.Signature(sig) as s:
        pk = s.generate_keypair()
        t0 = time.perf_counter()
        sig_data = s.sign(b"benchmark")
        t1 = time.perf_counter()
        results.append(("SIG", sig, len(pk), len(sig_data), 0, (t1 - t0) * 1e3))

df = pd.DataFrame(results, columns=["type","algo","pk_or_pkB","ct_or_sigB","ssB","ms"])
display(df)


In [ ]:
import os, matplotlib.pyplot as plt

out_dir = os.path.join("exercises", "pqc_demo", "images")
os.makedirs(out_dir, exist_ok=True)
out_path = os.path.join(out_dir, "pqc_timing.png")

plt.figure(figsize=(9,4.5))
labels = [r[1] for r in results]
times  = [r[5] for r in results]
bars = plt.bar(labels, times)
plt.xticks(rotation=20, ha="right")
plt.ylabel("Time (ms)")
plt.title("PQC ops timing (single-shot)")
for i, b in enumerate(bars):
    plt.text(b.get_x()+b.get_width()/2, b.get_height()+0.5, f"{times[i]:.1f}", ha="center", va="bottom", fontsize=9)
plt.tight_layout()
plt.savefig(out_path, dpi=160)
plt.show()
print("Saved chart to:", out_path)
